1. Трекинг объекта по цвету с использованием ползунков диапазона цвета в пространстве HSV
с маркировкой объекта ограничивающим прямоугольником.

In [ ]:
import cv2
import numpy as np

kernel = np.ones((5, 5), np.uint8)
cap = cv2.VideoCapture(0)

def nothing(x):
    pass
cv2.namedWindow('controls', cv2.WINDOW_NORMAL)

cv2.createTrackbar('H_lower','controls', 0, 179, nothing)
cv2.createTrackbar('S_lower','controls', 0, 255, nothing)
cv2.createTrackbar('V_lower','controls', 0, 255, nothing)
cv2.createTrackbar('H_upper','controls', 0, 179, nothing)
cv2.createTrackbar('S_upper','controls', 0, 255, nothing)
cv2.createTrackbar('V_upper','controls', 0, 255, nothing)

alive = True
while alive:
    
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)  #  делаем эффект зеркала
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    H_l = int(cv2.getTrackbarPos('H_lower', 'controls'))
    H_u = int(cv2.getTrackbarPos('H_upper', 'controls'))
    S_l = int(cv2.getTrackbarPos('S_lower', 'controls'))
    S_u = int(cv2.getTrackbarPos('S_upper', 'controls'))
    V_l = int(cv2.getTrackbarPos('V_lower', 'controls'))
    V_u = int(cv2.getTrackbarPos('V_upper', 'controls'))
    
    # define range of the color in HSV
    lower = np.array([H_l, S_l, V_l])
    upper = np.array([H_u, S_u, V_u])
    
    # Threshold the HSV image to get only the color
    mask = cv2.inRange(hsv, lower, upper)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    
    contours, h = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    for x in range(len(contours)):
        area = cv2.contourArea(contours[x])
        if area > 600:
            x, y, w, h = cv2.boundingRect(contours[x])
            frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            frame = cv2.rectangle(frame, (x, y), (x+60, y-25), (0, 0, 0), -1)
            cv2.putText(frame, 'Object', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
            frame = cv2.circle(frame, (x + (w//2), y + (h//2)), 15, (0, 255, 0), -1)
            print('x: ', (w//2), 'y: ', (h//2))  #фиксируем координаты объекта
    
    cv2.namedWindow('mask', cv2.WINDOW_NORMAL)
    cv2.imshow('mask', mask)
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(5) & 0xFF
    if key == ord('Q') or key == ord('q') or key == 27:
        alive = False

        

cap.release() 
cv2.destroyAllWindows()
cv2.waitKey(1)

In [1]:
# видео с примером работы алгоритма

from IPython.display import HTML
HTML("""
<video width=524 controls>
  <source src="vid.mp4" type="video/mp4">
</video>
""")